# 🧩 3.3 – Closures y Decoradores

Los **closures** y **decoradores** son mecanismos que permiten encapsular comportamiento y añadir funcionalidades sin modificar el código original de una función. Este patrón es esencial en Python para escribir código más limpio y reutilizable.

---
## 🎯 Objetivos
- Comprender cómo funcionan los **closures** (funciones internas que recuerdan su entorno).
- Crear decoradores personalizados paso a paso.
- Aplicar decoradores con y sin parámetros.
- Encadenar varios decoradores sobre una misma función.

In [1]:
print('✅ Notebook 3.3 – Closures y Decoradores listo para practicar.')

✅ Notebook 3.3 – Closures y Decoradores listo para practicar.


---
## 1️⃣ ¿Qué es un Closure?

Un **closure** ocurre cuando una función interna **recuerda las variables del entorno donde fue creada**, aunque se ejecute fuera de él.

Esto permite crear funciones configuradas dinámicamente (funciones que generan funciones).

In [2]:
def crear_multiplicador(factor):
    def multiplicar(x):
        return x * factor
    return multiplicar

por_dos = crear_multiplicador(2)
por_tres = crear_multiplicador(3)

print(por_dos(10))
print(por_tres(10))

20
30


✅ La función `multiplicar` recuerda el valor de `factor` incluso después de que `crear_multiplicador` haya terminado su ejecución.

---
## 2️⃣ Ejercicio 1 — Crear un closure contador

Crea una función `crear_contador()` que devuelva otra función llamada `incrementar()`.
Cada vez que se llame a `incrementar()`, debe sumar 1 al contador y devolver el valor actualizado.

💡 *Pista:* usa una variable local en `crear_contador()` y la palabra clave `nonlocal` dentro de `incrementar()`.

In [3]:
# Escribe aquí tu código...

### ✅ Solución propuesta

In [3]:
def crear_contador():
    cuenta = 0
    def incrementar():
        nonlocal cuenta
        cuenta += 1
        return cuenta
    return incrementar

contador = crear_contador()
print(contador())
print(contador())
print(contador())

1
2
3


✅ Este patrón es útil para mantener **estado interno** sin usar variables globales.

---
## 3️⃣ Decoradores: funciones que modifican funciones

Un **decorador** es una función que recibe otra función y devuelve una versión modificada de ella.

Permite añadir comportamiento (por ejemplo, logs, validaciones, medición de tiempo, etc.) sin alterar la definición original.

In [19]:
def logger(func):
    def envoltura(*args, **kwargs):
        print(f'📘 Ejecutando {func.__name__} con args={args}, kwargs={kwargs}')
        resultado = func(*args, **kwargs)
        print(f'✅ Resultado: {resultado}')
        return resultado
    return envoltura

@logger
def sumar(a, b):
    return a + b

valor = sumar(5, 7)
print(valor)

📘 Ejecutando sumar con args=(5, 7), kwargs={}
✅ Resultado: 12
12


✅ El símbolo `@logger` aplica el decorador directamente sobre la función. Equivale a `sumar = logger(sumar)`.

---
## 4️⃣ Ejercicio 2 — Medir tiempo de ejecución

Crea un decorador `medir_tiempo(func)` que:
- Imprima cuánto tarda en ejecutarse una función.
- Devuelva el resultado original.

💡 *Pista:* usa `time.time()` antes y después de ejecutar la función decorada.

In [ ]:
# Escribe aquí tu código...

import time

def medir_tiempo(func):
    def envoltura():
        inicio = time.time()
        resultado = func()
        fin = time.time()
        print(f'⏱ Tiempo: {fin - inicio:.5f} segundos')
        return resultado

    return envoltura


@medir_tiempo
def unafuncion():
    print('soy una funcion')
    return 'soy el resultado de func'


@medir_tiempo
def otrafuncion():
    print('soy otra funcion')
    return 'soy el resultado de otra funcion'


@logger
@medir_tiempo
def tarea_pesada():
    suma = sum(range(1_000_000))
    return suma

print(unafuncion())
print(otrafuncion())
print(tarea_pesada())


soy una funcion
⏱ Tiempo: 0.00020 segundos
soy el resultado de func
soy otra funcion
⏱ Tiempo: 0.00010 segundos
soy el resultado de otra funcion
📘 Ejecutando tarea_pesada con args=(), kwargs={}
✅ Resultado: 499999500000
⏱ Tiempo: 0.03651 segundos
499999500000


### ✅ Solución propuesta

In [22]:
import time

def medir_tiempo(func):
    def envoltura(*args, **kwargs):
        inicio = time.time()
        resultado = func(*args, **kwargs)
        fin = time.time()
        print(f'⏱ Tiempo: {fin - inicio:.5f} segundos')
        return resultado
    return envoltura

@medir_tiempo
def tarea_pesada():
    suma = sum(range(1_000_000))
    return suma

tarea_pesada()

⏱ Tiempo: 0.01970 segundos


499999500000

✅ Este decorador es útil para **benchmarking** o detección de cuellos de botella en código.

---
## 5️⃣ Decoradores con parámetros

Para pasar argumentos al decorador, se **anida una función adicional**.

### �� Ejercicio 3 — Decorador con nivel de log
Crea un decorador `log_nivel(nivel)` que permita especificar el nivel de log al aplicarlo, por ejemplo:
```python
@log_nivel('INFO')
def procesar():
    pass
```
Debe imprimir algo como: `INFO: ejecutando procesar()`

In [32]:
# �� Pista: anida una función que reciba func dentro de otra que reciba nivel.

def log_nivel(nivel):
    def decorador(func):
        def wrapper(*args, **kwargs):
            print(f'{nivel}: ejecutando {func.__name__}')
            resultado = func(*args, **kwargs)
            return resultado
        return wrapper
    return decorador
        
@log_nivel('INFO')
def soylafuncionadecorar(v,e,x):
    print(v,e,x)
    # el log no se implementa en la funcion ... lo inyecta el decorador


soylafuncionadecorar(1, 2, 3)
soylafuncionadecorar(2,3,5)
soylafuncionadecorar(3,5,3)



INFO: ejecutando soylafuncionadecorar
1 2 3
INFO: ejecutando soylafuncionadecorar
2 3 5
INFO: ejecutando soylafuncionadecorar
3 5 3


### ✅ Solución propuesta

In [25]:
def log_nivel(nivel):
    def decorador(func):
        def envoltura(*args, **kwargs):
            print(f'{nivel}: ejecutando {func.__name__}')
            return func(*args, **kwargs)
        return envoltura
    return decorador

@log_nivel('INFO')
def procesar():
    print('Procesando datos...')

procesar()

INFO: ejecutando procesar
Procesando datos...


✅ Este patrón es el que usan frameworks como **Flask**, **FastAPI** o **pytest** para manejar rutas, logs o hooks.

---
## 6️⃣ Ejercicio 4 — Encadenar decoradores

Aplica varios decoradores sobre una misma función (`@logger` y `@medir_tiempo`) y observa el orden de ejecución.

💡 *Pista:* el decorador más cercano a la función se ejecuta primero.

In [10]:
# Escribe aquí tu prueba con decoradores encadenados.

### ✅ Solución propuesta

In [37]:
# @logger
@medir_tiempo
def sumar_lento(a, b):
    time.sleep(0.5)
    return a + b

sumar_lento(3, 7)

sumar_lento(10,39)

⏱ Tiempo: 0.50008 segundos
⏱ Tiempo: 0.50008 segundos


49

✅ Los decoradores se ejecutan en **orden inverso** al que aparecen: el más cercano a la función se aplica primero.

In [39]:
frutas = ['manzana', 'pera']
decoradas = [f"🍎 {f}" for f in frutas]

print(decoradas)


['🍎 manzana', '🍎 pera']


---
## 🧠 Resumen del notebook

- Los **closures** permiten crear funciones con memoria interna.
- Los **decoradores** envuelven funciones para añadir comportamiento.
- Se pueden encadenar y parametrizar.
- Son una herramienta fundamental en frameworks modernos de Python.

💡 Próximo paso → **3.4 – Laboratorio de Funciones y Decoradores Aplicados.**